In [ ]:
import nbformat

nb = nbformat.read("ml_project.ipynb", as_version=4)

# Normalize the notebook to add missing id fields and other updates
nbformat.validator.validate(nb)

# Write the normalized notebook back to a file
nbformat.write(nb, "ml_project_normalized.ipynb")

%run ml_project_normalized.ipynb

Note: you may need to restart the kernel to use updated packages.
dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned color
cleaned breed
dropped columns


/var/folders/zg/knpz54sj16988z9vrp2b98fr0000gn/T/ipykernel_78902/1199118304.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_series = pd.to_datetime(df['intake_time'], errors='coerce')


cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned color
cleaned breed
Done running ml_project.ipynb.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import balanced_accuracy_score
import numpy as np
import pandas as pd

def generate_oof_predictions(model_fn, X, y, X_test, rare_classes, cat_cols, n_splits=5):
    """
    Generates out-of-fold predictions and test predictions from a given model training function.

    Returns:
      oof_preds: array of shape (n_samples, n_classes)
      test_preds: array of shape (n_test_samples, n_classes)
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    n_classes = len(np.unique(y))
    oof_preds = np.zeros((X.shape[0], n_classes))
    test_preds = np.zeros((X_test.shape[0], n_classes))

    for train_idx, valid_idx in skf.split(X, y):
        X_train_fold, y_train_fold = X.iloc[train_idx], y.iloc[train_idx]
        X_valid_fold, y_valid_fold = X.iloc[valid_idx], y.iloc[valid_idx]

        model, _ = model_fn(X_train_fold.copy(), y_train_fold.copy(), X_valid_fold.copy(), rare_classes, cat_cols)

        # Predict probabilities (not labels!)
        valid_probs = model.predict_proba(X_valid_fold)
        oof_preds[valid_idx] = valid_probs

        test_probs = model.predict_proba(X_test)
        test_preds += test_probs / n_splits  # average over folds

    return oof_preds, test_preds

In [ ]:
# This stacking took a long time to train, not sure if the code is set up for stacking correctly. Never were able to get a good
# set of predictions from stacking.
def stack_models(
    X_train, y_train, X_test,
    cat_model_fn, xgb_model_fn,
    cat_cols, rare_classes, n_splits=5
):
    """
        Generates predictions from stacked classifier components (catboost, xgboost) 
        and then stacks predictions On Logistic Regression model.

        Parameters:
            X_train: train dataset
            y_train: train dataset labels for validation
            X_test: test dataset
            cat_model_fn: trained catboost classifier
            xgb_model_fn: trained xgboost classifier
            cat_cols: the categorical columns in the dataset
            rare_classes: class labels that make up a small portion of the total dataset
            n_splits: number of folds to train over when generating out of fold predictions

        Returns:
            meta_model: trained, stacked log reg model
            meta_preds_test: test dataset predictions from model
        
    """
    print("[INFO] Generating OOF predictions for CatBoost...")
    cat_oof, cat_test = generate_oof_predictions(cat_model_fn, X_train, y_train, X_test, rare_classes, cat_cols, n_splits)
    
    print("[INFO] Generating OOF predictions for XGBoost...")
    xgb_oof, xgb_test = generate_oof_predictions(xgb_model_fn, X_train, y_train, X_test, rare_classes, cat_cols, n_splits)

    # Stack the base model outputs
    meta_X_train = np.hstack((cat_oof, xgb_oof))
    meta_X_test  = np.hstack((cat_test, xgb_test))

    # Train meta-model
    meta_model = LogisticRegression(max_iter=1000, multi_class='multinomial')
    meta_model.fit(meta_X_train, y_train)

    # Evaluate
    meta_preds_train = meta_model.predict(meta_X_train)
    bal_acc = balanced_accuracy_score(y_train, meta_preds_train)
    print(f"[INFO] Meta-model balanced accuracy on training OOF set: {bal_acc:.4f}")

    # Predict on test set
    meta_preds_test = meta_model.predict(meta_X_test)

    return meta_model, meta_preds_test


In [ ]:
# Step 1: Import train_classifier from both notebooks
print("[INFO] Importing model training functions from notebooks...")
%run xg_boost.ipynb
xgb_trainer = train_classifier

%run catboost_working.ipynb
catboost_trainer = train_classifier

X_train = df_train.drop(columns=['outcome_type'])
y_train = df_train['outcome_type']
X_test = df_test.copy()

# Step 2: Load or prepare your data (replace with actual data loading)
print("[INFO] Loading and preparing data...")
# Encode outcome labels
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
print('Encoding mapping:', dict(zip(le.classes_, le.transform(le.classes_))))

# Identify rare classes (<5% of samples)
rare_classes = [
    label for label, count in pd.Series(y_train_enc).value_counts().items()
    if count < 0.05 * len(y_train_enc)
]
print("\nRare classes:")
for cls in rare_classes:
    print(f"  {cls}: {le.classes_[cls]}")

# Define categorical columns
cat_cols = {'intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed', 'primary_color'}
categorical_features = [col for col in X_train.columns if col in cat_cols]

# Step 3: Run stacking
print("[INFO] Running stacking ensemble...")
meta_model, meta_test_preds = stack_models(
    X_train=X_train,
    y_train=y_train_enc,
    X_test=X_test,
    cat_model_fn=catboost_trainer,
    xgb_model_fn=xgb_trainer,
    cat_cols=categorical_features,
    rare_classes=rare_classes,
    n_splits=5
)

# Step 4: Evaluate and save predictions
decoded_preds = le.inverse_transform(meta_test_preds)
print(f"[INFO] Sample predictions: {decoded_preds[:5]}")

save_predictions(decoded_preds, model_name="stacked_cat_xgb_logreg")

[INFO] Importing model training functions from notebooks...
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned color
cleaned breed
dropped columns


/var/folders/zg/knpz54sj16988z9vrp2b98fr0000gn/T/ipykernel_78902/1199118304.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_series = pd.to_datetime(df['intake_time'], errors='coerce')


cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned color
cleaned breed
Done running ml_project.ipynb.
[4 6 'Common' 0 'Rare' 1 7 5 8 2 3 'Unknown']
Encoding mapping: ['Adoption' 'Died' 'Euthanasia' 'Return to Owner' 'Transfer']
Rare classes:
  2: Euthanasia
  1: Died

[INFO] Starting training with 111155 samples and 111155 labels
Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END xgb__colsample_bytree=0.5621780831931538, xgb__gamma=0.9507143064099162, xgb__learning_rate=0.22227824312530747, xgb__max_depth=6, xgb__min_child_weight=7, xgb__n_estimators=171, xgb__reg_alpha=0.15599452033620265, xgb__reg_lambda=0.6452090304204987, xgb__subsample=0.9063233020424546;, score=0.536 total time=  41.2s
[CV 2/5] END xgb__colsample_bytree=0.5621780831931538, xgb__gamma=0.9507143064099162, xgb__learning_rate=0.22227824312530747, xgb__max_depth=6, xgb__min_child_weight=7, xgb__n_estimators=171, xgb__reg_alpha=0.15599452033620265, xgb__reg_lambda=0.6452090